<font color="orange">Multilevel Monte Carlo simulation for VIX options in the rough
Bergomi model</font>

Convention is different here: the parameter $\eta$ in the paper must be 
replaced with $\eta \sqrt{2H}$

Let $\Delta = 30$ days. The VIX is defined as 

$$
    \text{VIX}_T = \sqrt{\frac1\Delta \int_{T}^{T+\Delta} \xi_T^u \, du}
$$

where 

$$
    \xi_T^u = \xi_0^u 
    \exp\left\{ \eta Y_T^u - \eta^2(u^{2H} - (u-T)^{2H})\right\}
$$

and 

$$
    Y_t^u = \sqrt{2H} \int_0^t (u-s)^{H-1/2} \, dW_s, \qquad \forall u \geq t.
$$

We need to compute the covariance matrix of the vector 
$(Y_T^{u_i})_{0 \leq i \leq n}$ where $u_i = T + i \Delta/n$ for $i = 0, 1, \ldots, n$.

In [1]:
%load_ext autoreload
%autoreload 2
%config InlineBackend.figure_format = 'retina'

In [2]:
import matplotlib.pyplot as plt
from rbergomi import RoughBergomi
import matplotlib as mpl
import numpy as np
import seaborn as sns

sns.set_theme("talk")
mpl.rcParams["figure.figsize"] = (8, 6)

For $n \in \mathbb{N}^*$, consider the uniform grid $u_i = T + i \Delta/n$ for $i = 0, 1, \ldots, n$.

We are interested in the right-point rectangle scheme

$$
    \text{VIX}_T^{2,\mathcal{R}_n}
    = \frac{1}{n} \sum_{i=1}^{n} \xi_T^{u_i}
$$

and the trapezoidal scheme

$$
    \text{VIX}_T^{2,\mathcal{T}_n}
    = \frac{1}{2n} \sum_{i=1}^{n} 
    \left(\xi_T^{u_{i-1}} + \xi_T^{u_i}\right)
$$

Observe that $\{\log \xi_{T}^{u_i}\}_{0 \leq i \leq n}$
forms an $(n+1)$-dimensional Gaussian vector with mean vector 
$\mu = \{\mu_i\}_{0 \leq i \leq n}$ and covariance matrix $C = \{C_{i,j}\}_{0 \leq i,j \leq n}$
defined as

$$
    \mu_i = \log \xi_0^{u_i} - \frac{\eta^2}{2} (u_i^{2H} - (u_i-T)^{2H})
$$

and 

$$
    C_{i,j} = 2 H \int_0^T (u_i-s)^{H-1/2} (u_j-s)^{H-1/2} \, ds
$$

The covariance matrix term $C_{i,j}$ can be computed using the Gaussian 
hypergeometric function.

A Monte Carlo estimator for the price of a VIX option with payoff $\varphi$ is

$$
    \mathbb{E}[\varphi(\text{VIX}_T^{2})] 
    \approx \frac{1}{M} \sum_{m=1}^{M} 
    \varphi \left(\frac{1}{n} \sum_{i=1}^{n} \exp\{X_T^{u_i,m}\}\right)
$$

where $X_T^u = \log \xi_T^u$. 

A natural control variate is the lognormal random variable
$\exp\left\{\frac{1}{n} \sum_{i=1}^{n}X_T^{u_i} \right\}$. Observe that
$\frac{1}{n} \sum_{i=1}^{n}X_T^{u_i}$ is normal with mean and standard deviation

$$
    \bar{\mu}_n = \frac{1}{n} \sum_{i=1}^{n} \mu_i,
    \qquad
    \bar{\sigma}_n = \sqrt{\frac{1}{n^2} \sum_{i,j=1}^{n} C_{i,j}}
$$




# Strong error

# Weak error

# Multilevel Monte Carlo